In [ ]:
import sys

# Add the parent directory to the system path
sys.path.append("../04_survival_models/src")

In [ ]:
import datetime
import json
import os
import pickle
import pprint
import time
import warnings

import joblib
import kaplanmeier as km
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import optuna
import pandas as pd
from azureml.core import Dataset, Workspace
from lifelines.statistics import logrank_test
from sklearn.experimental import enable_iterative_imputer
from sklearn.feature_selection import SelectKBest
from sklearn.impute import IterativeImputer
from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    ParameterGrid,
    RandomizedSearchCV,
    StratifiedKFold,
    train_test_split,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sksurv.ensemble import RandomSurvivalForest
from sksurv.functions import StepFunction
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import (
    concordance_index_censored,
    concordance_index_ipcw,
    cumulative_dynamic_auc,
    integrated_brier_score,
)
from sksurv.nonparametric import kaplan_meier_estimator
from uc2_functions import *
from tqdm import tqdm

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Goal

The goal is to fine-tune a Cox model using GRANT features as the baseline and to tune a RandomSurvivalForest with all available features as the feature selector.

# Parameters

In [ ]:
# Legend
PATH_LEGEND = "Legenda_Variabili_Uri_Larcher.xlsx"
# Directories
DIR_SC = os.path.join(os.path.dirname(os.getcwd()), "sc")  # Legend
DIR_MODEL_JSON = "../models_json"  # Parameters for the models not used during inference
DIR_MODEL_PKL = "../models_pkl"  # Weights for the models used during inference

In [ ]:
RANDOM_STATE = 42
EXPERIMENT_NAME = "UC2_larcher_2024_02"
PARENT_RUN_ID = None

# Data ingestion

## One-hot encoding version

In [ ]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required

subscription_id = "753a0b42-95dc-4871-b53e-160ceb0e6bc1"
resource_group = "rg-s-race-aml-dev-we"
workspace_name = "amlsraceamldevwe01"

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name="UC2_larcher_survival_csm_ohe_5yrs")
df_ohe = dataset.to_pandas_dataframe()
print(df_ohe.shape)
df_ohe.head()

### Use schema

Recreate the schema from tags:

In [ ]:
tags = dataset.tags

dtypes = json.loads(tags["dtypes_json"])
is_ordinal = json.loads(tags["is_ordinal_json"])

for col in dtypes.keys():
    if dtypes[col] == "category":
        categories = (
            sorted(df_ohe[col].dropna().unique())
            if is_ordinal[col]
            else df_ohe[col].dropna().unique()
        )
        df_ohe[col] = pd.Categorical(
            df_ohe[col], categories=categories, ordered=is_ordinal[col]
        )
    else:
        df_ohe[col] = df_ohe[col].astype(dtypes[col])

In [ ]:
count_columns_by_dtype(df_ohe)

## `.xlsx` Legend

In [ ]:
df_legend = pd.read_excel(
    os.path.join(DIR_SC, PATH_LEGEND), sheet_name="Legenda Urologi - DB Larcher"
)
# Replace dot with underscore
df_legend["Variable"] = df_legend["Variable"].apply(lambda x: x.replace(".", "_"))
# Forward fill domain
df_legend["Domain"] = df_legend["Domain"].fillna(method="ffill")
df_legend.head()

### Create dictionary

In [ ]:
dict_legend = pd.Series(
    df_legend["Definition"].values, index=df_legend["Variable"]
).to_dict()

# Start mlflow run

In [ ]:
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.start_run(run_name=str(RANDOM_STATE))
if PARENT_RUN_ID:
    mlflow.set_tag("parent_run_id", PARENT_RUN_ID)

# Drop na on target columns

In [ ]:
not_features = ["P_1_id", "death", "csm", "ocm", "ttdeath"]

In [ ]:
print(df_ohe.shape[0])
df_ohe = df_ohe.dropna(subset=["ttdeath", "death"])
print(df_ohe.shape[0])

# Censoring rate

In [ ]:
# Should be impossible to have ocm if ttdeath is < 60
df_ohe[df_ohe["ttdeath"] < 60]["ocm"].value_counts(dropna=False)

In [ ]:
# Distribution of ttdeath
df_ohe["ttdeath"].hist(bins=60)

In [ ]:
# Distribution of ttdeath, exluding 60
df_ohe[df_ohe["ttdeath"] < 60]["ttdeath"].hist(bins=59)

In [ ]:
# Distribution of ttdeath by binary class, exluding 60
fig, ax = plt.subplots(2, 1, figsize=(10, 8))

# Plot 1: death == False
ax[0].hist(df_ohe[(df_ohe["ttdeath"] < 60) & (df_ohe["death"] == False)]["ttdeath"], bins=60)
ax[0].set_title('Non morti + censurati')
ax[0].set_xlabel('ttdeath')
ax[0].set_ylabel('Frequency')

# Plot 2: death == True
ax[1].hist(df_ohe[(df_ohe["ttdeath"] < 60) & (df_ohe["death"] == True)]["ttdeath"], bins=60)
ax[1].set_title('Morti')
ax[1].set_xlabel('ttdeath')
ax[1].set_ylabel('Frequency')

# General title
fig.suptitle('DBURI Dataset', fontsize=16)

plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to fit the suptitle
plt.show()

In [ ]:
# Censoring rate
print(df_ohe["death"].value_counts(normalize=True))

Harrell’s concordance index is known to be biased upwards if the amount of censoring in the test data is high. Uno et al proposed an alternative estimator of the concordance index that behaves better in such situations. Therefore, we are going to apply `concordance_index_ipcw` as the main metric and `concordance_index_censored` as a reference.

# Train test split

## List features

In [ ]:
features_all = sorted(set(df_ohe.columns.tolist()) - set(not_features))
print(len(features_all))

## Train test split

In [ ]:
# Define features and target
X = df_ohe[features_all]
y = np.array(
    [(event, time) for event, time in zip(df_ohe["death"], df_ohe["ttdeath"])],
    dtype=[("event", bool), ("time", float)],
)
ids = df_ohe["P_1_id"]
mlflow.log_param(
    "death_perc_5yrs",
    pd.Series(y["event"]).value_counts(sort=True, normalize=True)[True],
)

# Split data and IDs into training and testing sets
(
    X_train_missing,
    X_test_missing,
    y_train,
    y_test,
    ids_train,
    ids_test,
) = train_test_split(
    X,
    y,
    ids,
    test_size=0.2,
    stratify=y["event"],
    random_state=RANDOM_STATE,
)
del X, y, ids
# Check distributions of death event on train and test
print(pd.Series(y_train["event"]).value_counts(sort=True, normalize=True))
print(pd.Series(y_test["event"]).value_counts(sort=True, normalize=True))

# Imputation

## Fit and trasform on train

In [ ]:
X_train = X_train_missing.copy()

imputer = IterativeImputer(
    max_iter=25, initial_strategy="median", random_state=RANDOM_STATE
)
imputer = imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_train = pd.DataFrame(X_train, columns=X_train_missing.columns)

# Assert
assert set(X_train.columns) == set(X_train_missing.columns)

del X_train_missing

## Transform on test

In [ ]:
X_test = X_test_missing.copy()

X_test = imputer.transform(X_test)
X_test = pd.DataFrame(X_test, columns=X_test_missing.columns)

# Assert
assert set(X_test.columns) == set(X_test_missing.columns)

del X_test_missing

# Cox model - GRANT fine-tune

In [ ]:
model_name = "CoxPHSurvivalAnalysis_grant_finetune_T1"
mlflow.start_run(run_name=model_name, nested=True)
mlflow.log_param("random_state", RANDOM_STATE)

As baseline we train a Cox model trained on features from prognostic model GRANT (table Table 6.3 at https://uroweb.org/guidelines/renal-cell-carcinoma/chapter/prognostic-factors):

1. AGE

2. T classification

3. N classification

4. (Fuhrman) grade

In [ ]:
features_grant = [
    "age",
    "pT",
    "pN_1_0",
    "grade",
]

## Train

In [ ]:
# Train the model
cox_grant = CoxPHSurvivalAnalysis()
cox_grant.fit(X_train[features_grant], y_train)
mlflow.log_param("feature_names_in", cox_grant.feature_names_in_)
mlflow.log_param("n_features_in", cox_grant.n_features_in_)

## Save model weights to pkl

In [ ]:
# Save model weights to pkl
os.makedirs(DIR_MODEL_PKL, exist_ok=True)
model_path = os.path.join(DIR_MODEL_PKL, "larcher_{}_{}.pkl".format(model_name, RANDOM_STATE))
joblib.dump(cox_grant, model_path)
mlflow.log_artifact(model_path)
mlflow.log_param("model_path", model_path)

## Validate

In [ ]:
result_censored, result_ipcw, score_brier, mean_auc, fig = validate_sksurv_model(model=cox_grant,
                                                                                 y_train=y_train,
                                                                                 X_test=X_test[features_grant],
                                                                                 y_test=y_test,
                                                                                 tau=60)
print("concordance_index_censored", round(result_censored, 3))
mlflow.log_metric("concordance_index_censored", result_censored)
print("concordance_index_ipcw", round(result_ipcw, 3))
mlflow.log_metric("concordance_index_ipcw", result_ipcw)
print("integrated_brier_score", round(score_brier, 3))
mlflow.log_metric("integrated_brier_score", score_brier)
print("mean_cumulative_dynamic_auc", round(mean_auc, 3))
mlflow.log_metric("mean_cumulative_dynamic_auc", mean_auc)
mlflow.log_figure(fig, "time_dependent_auc.png")
plt.show(fig)

In [ ]:
mlflow.end_run()

# Random Survival Forest - T1

In [ ]:
model_name = "RandomSurvivalForest_selector_T1"
mlflow.start_run(run_name=model_name, nested=True)
mlflow.log_param("random_state", RANDOM_STATE)

The goal is to obtain a ranking of important features at t1 to gain medical knowledge, and use it as imput to a Cox model (feature selection).

## Hyperparameter search with Optuna

In [ ]:
print(np.log2(len(features_all)))
print(np.sqrt(len(features_all)))

In [ ]:
# Define search spaces
grid = {
    "n_estimators": (10, 100),
    "max_depth": (2, 50),
    "min_samples_split": (2, 50),
    "min_samples_leaf": (2, 50),
    "max_features": (8, min(50, len(features_all))),
}

In [ ]:
# Hyperparameter search with Optuna
optimize_rsf(X_tune=X_train,
             y_tune=y_train,
             grid=grid,
             n_trials=1000,
             n_folds=10,
             model_dir=DIR_MODEL_JSON,
             model_filename="larcher_{}_{}.json".format(model_name, RANDOM_STATE),
             random_state=RANDOM_STATE)

## Fit best model

In [ ]:
# Define the file path
file_path = os.path.join(
    DIR_MODEL_JSON, "larcher_{}_{}.json".format(model_name, RANDOM_STATE)
)

# Read the JSON file to get the best hyperparameters
with open(file_path, "r") as f:
    data = json.load(f)
    print("Log from best RandomSurvivalForest model:")
    pprint.pprint(data)
    mlflow.log_params(data)
    print()
    print("Grid:")
    pprint.pprint(grid)
    assert data["random_state"] == RANDOM_STATE
    mlflow.log_params(grid)

# Create a new RandomSurvivalForest instance with the best parameters
rsf_best_t1 = RandomSurvivalForest(**data["best_params"], random_state=RANDOM_STATE)

# Fit the model on the complete training data
rsf_best_t1.fit(X_train, y_train)
if len(rsf_best_t1.feature_names_in_) < 50:
    mlflow.log_param("feature_names_in", rsf_best_t1.feature_names_in_)
else:
    mlflow.log_param("feature_names_in", "More than 50 features")
mlflow.log_param("n_features_in", rsf_best_t1.n_features_in_)

## Validate

In [ ]:
result_censored, result_ipcw, score_brier, mean_auc, fig = validate_sksurv_model(model=rsf_best_t1,
                                                                                 y_train=y_train,
                                                                                 X_test=X_test,
                                                                                 y_test=y_test,
                                                                                 tau=60)
print("concordance_index_censored", round(result_censored, 3))
mlflow.log_metric("concordance_index_censored", result_censored)
print("concordance_index_ipcw", round(result_ipcw, 3))
mlflow.log_metric("concordance_index_ipcw", result_ipcw)
print("integrated_brier_score", round(score_brier, 3))
mlflow.log_metric("integrated_brier_score", score_brier)
print("mean_cumulative_dynamic_auc", round(mean_auc, 3))
mlflow.log_metric("mean_cumulative_dynamic_auc", mean_auc)
mlflow.log_figure(fig, "time_dependent_auc.png")
plt.show(fig)

## Feature importance

- Training Set: Conducting feature importance analysis on the training set provides insights into the features that the model has learned. However, this approach may not offer an accurate representation of how these features generalize to new data. This allows to use the features as input for other models without leakage.

- Test Set: Analyzing feature importance on the test set offers an understanding of feature performance on unseen data. Nonetheless, the reliability of this method may be compromised if the test set is small and imbalanced.

### Train set

In [ ]:
result_rsf_t1 = permutation_importance(
    rsf_best_t1, X_train, y_train, n_repeats=50, n_jobs=-1, random_state=RANDOM_STATE
)

# Crate dataframe
df_importance_rsf_t1 = pd.DataFrame(
    {
        k: result_rsf_t1[k]
        for k in (
            "importances_mean",
            "importances_std",
        )
    },
    index=X_train.columns,
).sort_values(by="importances_mean", ascending=False)
del result_rsf_t1

# Get definition from dictionary
df_importance_rsf_t1["feature_definition"] = df_importance_rsf_t1.index
df_importance_rsf_t1["feature_definition"] = df_importance_rsf_t1["feature_definition"].apply(
    lambda x: replace_longest_match(x, dict_legend)
)
mlflow.log_dict(df_importance_rsf_t1.to_dict(), "df_importance_rsf")

In [ ]:
plot_feature_importance(df_importance_rsf_t1, 15, (5, 8))

Is it better to use 8 levels or 4 levels on `pT`?
```
mapping_t_4lev = {
    "T1a": 1.0,
    "T1b": 1.0,
    "T2a": 2.0,
    "T2b": 2.0,
    "T3a": 3.0,
    "T3b": 3.0,
    "T3c": 3.0,
    "T4": 4.0,
    "Tx": np.nan,
}  # Rare event

mapping_t_8lev = {
    "T1a": 1.0,
    "T1b": 2.0,
    "T2a": 3.0,
    "T2b": 4.0,
    "T3a": 5.0,
    "T3b": 6.0,
    "T3c": 7.0,
    "T4": 8.0,
    "Tx": np.nan,
}  # Rare event
```

In [ ]:
print(df_importance_rsf_t1.index.tolist().index("pT"))  # 8 levels
print(df_importance_rsf_t1.index.tolist().index("pT_4lev"))  # 4 levels

Which of the GRANT features (baseline) are included in the top k features of the Random Survival Forest?

In [ ]:
k = int(len(df_importance_rsf_t1) * 0.25)  # First quartile
print(k)
# Top k features as list (order of importance)
features_selected_raw = df_importance_rsf_t1.index.tolist()[:k]

for x in features_grant:
    print(x)
    if x in features_selected_raw:
        print(
            "\tIncluded in top {} features for Random Survival Forest (rank {})".format(
                k, features_selected_raw.index(x)
            )
        )
        print("\n")
    else:
        print("\n")

In [ ]:
mlflow.end_run()

# Random Survival Forest - T0

In [ ]:
model_name = "RandomSurvivalForest_selector_T0"
mlflow.start_run(run_name=model_name, nested=True)
mlflow.log_param("random_state", RANDOM_STATE)

The goal is to obtain a ranking of important features at t0 to gain medical knowledge, and use it as imput to a Cox model (feature selection).

In [ ]:
# Features from the legend
temp_0 = df_legend[df_legend["Domain"] == "clinical history"]["Variable"].tolist()

# Check if the columns in legend are present in actual dataframe
features_t0 = []
for feature in temp_0:
    for col in df_ohe:
        # Check if the actual column starts with the feature (followed by an underscore or nothing)
        if col.startswith(f"{feature}_") or col == feature:
            features_t0.append(col)
del temp_0
print(len(features_t0))

## Hyperparameter search with Optuna

In [ ]:
# Define search spaces
grid = {
    "n_estimators": (10, 100),
    "max_depth": (2, 50),
    "min_samples_split": (2, 50),
    "min_samples_leaf": (2, 50),
    "max_features": (8, min(50, len(features_t0))),
}

In [ ]:
# Hyperparameter search with Optuna
optimize_rsf(X_tune=X_train[features_t0],
             y_tune=y_train,
             grid=grid,
             n_trials=1000,
             n_folds=10,
             model_dir=DIR_MODEL_JSON,
             model_filename="larcher_{}_{}.json".format(model_name, RANDOM_STATE),
             random_state=RANDOM_STATE)

## Fit best model

In [ ]:
# Define the file path
file_path = os.path.join(
    DIR_MODEL_JSON, "larcher_{}_{}.json".format(model_name, RANDOM_STATE)
)

# Read the JSON file to get the best hyperparameters
with open(file_path, "r") as f:
    data = json.load(f)
    print("Log from best RandomSurvivalForest model:")
    pprint.pprint(data)
    mlflow.log_params(data)
    print()
    print("Grid:")
    pprint.pprint(grid)
    assert data["random_state"] == RANDOM_STATE
    mlflow.log_params(grid)

# Create a new RandomSurvivalForest instance with the best parameters
rsf_best_t0 = RandomSurvivalForest(**data["best_params"], random_state=RANDOM_STATE)

# Fit the model on the complete training data
rsf_best_t0.fit(X_train[features_t0], y_train)
if len(rsf_best_t0.feature_names_in_) < 50:
    mlflow.log_param("feature_names_in", rsf_best_t0.feature_names_in_)
else:
    mlflow.log_param("feature_names_in", "More than 50 features")
mlflow.log_param("n_features_in", rsf_best_t0.n_features_in_)

## Validate

In [ ]:
result_censored, result_ipcw, score_brier, mean_auc, fig = validate_sksurv_model(model=rsf_best_t0,
                                                                                 y_train=y_train,
                                                                                 X_test=X_test[features_t0],
                                                                                 y_test=y_test,
                                                                                 tau=60)
print("concordance_index_censored", round(result_censored, 3))
mlflow.log_metric("concordance_index_censored", result_censored)
print("concordance_index_ipcw", round(result_ipcw, 3))
mlflow.log_metric("concordance_index_ipcw", result_ipcw)
print("integrated_brier_score", round(score_brier, 3))
mlflow.log_metric("integrated_brier_score", score_brier)
print("mean_cumulative_dynamic_auc", round(mean_auc, 3))
mlflow.log_metric("mean_cumulative_dynamic_auc", mean_auc)
mlflow.log_figure(fig, "time_dependent_auc.png")
plt.show(fig)

## Feature importance

### Train set

In [ ]:
result_rsf_t0 = permutation_importance(
    rsf_best_t0,
    X_train[features_t0],
    y_train,
    n_repeats=50,
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

# Crate dataframe
df_importance_rsf_t0 = pd.DataFrame(
    {
        k: result_rsf_t0[k]
        for k in (
            "importances_mean",
            "importances_std",
        )
    },
    index=X_train[features_t0].columns,
).sort_values(by="importances_mean", ascending=False)
del result_rsf_t0

# Get definition from dictionary
df_importance_rsf_t0["feature_definition"] = df_importance_rsf_t0.index
df_importance_rsf_t0["feature_definition"] = df_importance_rsf_t0[
    "feature_definition"
].apply(lambda x: replace_longest_match(x, dict_legend))
mlflow.log_dict(df_importance_rsf_t0.to_dict(), "df_importance_rsf")

In [ ]:
plot_feature_importance(df_importance_rsf_t0, 15, (5, 8))

In [ ]:
mlflow.end_run()

# End mlflow run

In [ ]:
mlflow.end_run()